In [1]:
import time
import torch

from rcpsp_inst import load_dataset
from net import Net
from aco import ACO_RCPSP

lr = 1e-3
EPS = 1e-10
ALPHA = 0.05
T=20
device = 'cpu'

In [2]:
@torch.no_grad()
def infer_instance(model, rcpsp, n_ants, t_aco_diff):
    if model:
        model.eval()
        pyg_data = rcpsp.to_pyg_data()
        phe_vec, heu_vec = model(pyg_data, require_phe=True, require_heu=True)
        phe_mat, heu_mat = model.reshape(pyg_data, phe_vec) + EPS, model.reshape(pyg_data, heu_vec) + EPS
    
        aco = ACO_RCPSP(
            rcpsp,
            n_ants=n_ants,
            pheromone=phe_mat,
            heuristic=heu_mat,
            device=device
        )
    
    else:
        aco = ACO_RCPSP(
            rcpsp,
            n_ants=n_ants,
            device=device
        )
        
    results = torch.zeros(size=(len(t_aco_diff),), device=device)
    for i, t in enumerate(t_aco_diff):
        best_cost = aco.run(t).cost
        results[i] = best_cost
    return results
        
    
@torch.no_grad()
def test(dataset, model, n_ants, t_aco):
    _t_aco = [0] + t_aco
    t_aco_diff = [_t_aco[i+1]-_t_aco[i] for i in range(len(_t_aco)-1)]
    sum_results = torch.zeros(size=(len(t_aco_diff),), device=device)
    start = time.time()
    for instance in dataset:
        results = infer_instance(model, instance, n_ants, t_aco_diff)
        sum_results += results
    end = time.time()
    
    return sum_results / len(dataset), end-start

### Test on j30

In [3]:
n_ants = 20
n_node = 30
t_aco = [1, 10, 20, 30, 40, 50, 100]
_, test_list = load_dataset(f"../data/rcpsp/j{n_node}rcp")

In [4]:
# MetaACO
net_tsp = Net().to(device)
net_tsp.load_state_dict(torch.load(f'../pretrained/rcpsp/rcpsp{n_node}-5.pt', map_location=device))
avg_aco_best, duration = test(test_list, net_tsp, n_ants, t_aco)
print('total duration: ', duration)
for i, t in enumerate(t_aco):
    print("T={}, average cost is {}.".format(t, avg_aco_best[i]))    

total duration:  206.4487235546112
T=1, average cost is 61.95000076293945.
T=10, average cost is 59.189998626708984.
T=20, average cost is 58.54999923706055.
T=30, average cost is 58.130001068115234.
T=40, average cost is 57.86000061035156.
T=50, average cost is 57.779998779296875.
T=100, average cost is 57.41999816894531.


In [5]:
# ACO
avg_aco_best, duration = test(test_list, None, n_ants, t_aco)
print('total duration: ', duration)
for i, t in enumerate(t_aco):
    print("T={}, average cost is {}.".format(t, avg_aco_best[i]))

total duration:  202.41569781303406
T=1, average cost is 63.79999923706055.
T=10, average cost is 60.650001525878906.
T=20, average cost is 60.0.
T=30, average cost is 59.720001220703125.
T=40, average cost is 59.599998474121094.
T=50, average cost is 59.529998779296875.
T=100, average cost is 59.43000030517578.


### Test on j60

In [6]:
n_ants = 20
n_node = 60
t_aco = [1, 10, 20, 30, 40, 50, 100]
_, test_list = load_dataset(f"../data/rcpsp/j{n_node}rcp")

In [7]:
# MetaACO
net_tsp = Net().to(device)
net_tsp.load_state_dict(torch.load(f'../pretrained/rcpsp/rcpsp{n_node}-5.pt', map_location=device))
avg_aco_best, duration = test(test_list, net_tsp, n_ants, t_aco)
print('total duration: ', duration)
for i, t in enumerate(t_aco):
    print("T={}, average cost is {}.".format(t, avg_aco_best[i]))

total duration:  395.4517147541046
T=1, average cost is 99.5999984741211.
T=10, average cost is 92.94999694824219.
T=20, average cost is 91.80999755859375.
T=30, average cost is 90.83000183105469.
T=40, average cost is 90.37000274658203.
T=50, average cost is 89.70999908447266.
T=100, average cost is 88.61000061035156.


In [8]:
# ACO
avg_aco_best, duration = test(test_list, None, n_ants, t_aco)
print('total duration: ', duration)
for i, t in enumerate(t_aco):
    print("T={}, average cost is {}.".format(t, avg_aco_best[i]))

total duration:  395.55902886390686
T=1, average cost is 99.68000030517578.
T=10, average cost is 93.83999633789062.
T=20, average cost is 92.81999969482422.
T=30, average cost is 92.41999816894531.
T=40, average cost is 92.26000213623047.
T=50, average cost is 92.13999938964844.
T=100, average cost is 92.05999755859375.


### Test on j120

In [9]:
n_ants = 20
n_node = 120
t_aco = [1, 10, 20, 30, 40, 50, 100]
_, test_list = load_dataset(f"../data/rcpsp/j{n_node}rcp")

In [10]:
# MetaACO
net_tsp = Net().to(device)
net_tsp.load_state_dict(torch.load(f'../pretrained/rcpsp/rcpsp{n_node}-5.pt', map_location=device))
avg_aco_best, duration = test(test_list, net_tsp, n_ants, t_aco)
print('total duration: ', duration)
for i, t in enumerate(t_aco):
    print("T={}, average cost is {}.".format(t, avg_aco_best[i]))    

total duration:  798.9243993759155
T=1, average cost is 196.2100067138672.
T=10, average cost is 186.63999938964844.
T=20, average cost is 185.22999572753906.
T=30, average cost is 184.02000427246094.
T=40, average cost is 183.32000732421875.
T=50, average cost is 182.69000244140625.
T=100, average cost is 180.69000244140625.


In [11]:
# ACO
avg_aco_best, duration = test(test_list, None, n_ants, t_aco)
print('total duration: ', duration)
for i, t in enumerate(t_aco):
    print("T={}, average cost is {}.".format(t, avg_aco_best[i]))

total duration:  795.6401858329773
T=1, average cost is 194.83999633789062.
T=10, average cost is 186.25.
T=20, average cost is 184.5.
T=30, average cost is 183.8000030517578.
T=40, average cost is 183.44000244140625.
T=50, average cost is 183.30999755859375.
T=100, average cost is 182.97999572753906.
